In [1]:
import os
import sys
current_dir = os.getcwd()
root_dir = os.path.dirname(os.path.dirname(current_dir))     # neural-lam
sys.path.append(root_dir)

import random
import torch
import pytorch_lightning as pl
from lightning_fabric.utilities import seed
from argparse import ArgumentParser
import time
import matplotlib.pyplot as plt
import wandb

from neural_lam.models.graph_lam import GraphLAM
from neural_lam.models.hi_lam import HiLAM
from neural_lam.models.hi_lam_parallel import HiLAMParallel

from neural_lam.weather_dataset import WeatherDataset
from neural_lam import constants, utils

# load model

In [6]:
model = GraphLAM.load_from_checkpoint('/root/Desktop/machine_learning/neural-lam/saved_models/graph_lam-4x64-12_04_20-1581/min_val_loss.ckpt')
model.eval()

Loaded graph with 70345 nodes (63784 grid, 6561 mesh)
Edges in subgraphs: m2m=57616, g2m=100656, m2g=255136


GraphLAM(
  (loss): MSELoss()
  (grid_embedder): Sequential(
    (0): Linear(in_features=54, out_features=64, bias=True)
    (1): SiLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  )
  (g2m_embedder): Sequential(
    (0): Linear(in_features=3, out_features=64, bias=True)
    (1): SiLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  )
  (m2g_embedder): Sequential(
    (0): Linear(in_features=3, out_features=64, bias=True)
    (1): SiLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  )
  (g2m_gnn): InteractionNet()
  (encoding_grid_mlp): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): SiLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  )
  (m2g_g

# load data

In [5]:
args = {}
args['step_length'] = 3
args['dataset'] = 'meps_example'
args['batch_size'] = 1
max_pred_length = (65 // args['step_length']) - 2 # 19
test_loader = torch.utils.data.DataLoader(
        WeatherDataset(args['dataset'], pred_length=max_pred_length, split="test",
            subsample_step=args['step_length'], subset=0,
            control_only=0),
        args['batch_size'], shuffle=False, num_workers=0)

In [7]:
test_dataset = WeatherDataset(args['dataset'], pred_length=max_pred_length, split="test",
            subsample_step=args['step_length'], subset=0,
            control_only=0)

# define trainer

In [15]:
trainer = pl.Trainer(accelerator='gpu', devices = [0])
predictions = trainer.predict(model=model, dataloaders=test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
/root/Desktop/machine_learning/neural-lam/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=255` in the `DataLoader` to improve performance.


Predicting DataLoader 0:   0%|          | 0/4 [00:00<?, ?it/s]

TypeError: predict_step() missing 2 required positional arguments: 'batch_static_features' and 'forcing'

In [10]:
pred = model(s)

NotImplementedError: Module [GraphLAM] is missing the required "forward" function